In [267]:
!pip install pinecone

In [268]:
from pinecone import Pinecone

pc = Pinecone("pcsk_5zSDm7_FiXyttnjXKArVHUKTq2wVnvigTxJD4jQtavV9MJ2kVQbVVjtJuso6pTUkvaJU2W")

# Embed data
data = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
]

embeddings = pc.inference.embed(
    model="llama-text-embed-v2",
    inputs=[d['text'] for d in data],
    parameters={
        "input_type": "passage"
    }
)

vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })
index = pc.Index('llama-text-embed-v2-index')





In [269]:
import getpass
import os


os.environ["GROQ_API_KEY"] ="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"

In [270]:
pip install -qU langchain-groq

In [271]:
user_info={"user_id":123}

In [272]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [273]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

The translation of "I love programming" to French is:

"J'adore le programmation."


In [274]:
from langchain.prompts import ChatPromptTemplate

controller_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a controller agent for a medical chatbot. Your job is to decide which of the following agents should handle the user query:

- Response_agent: Use this for general medical questions that do not require any personal user information. For example, if the user asks for general facts or opinions unrelated to their personal condition or history.

- retriever_agent: Use this when the user asks for medical advice or recommendations that depend on their personal data. If the user's own health, symptoms, or history are relevant, select this agent.

Reply with only one of the following:
- Response_agent
- retriever_agent
"""),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")  # This is the correct role for tool responses or internal reasoning
])


In [275]:
from langchain.agents import create_tool_calling_agent
tools=[]

controller_agent = create_tool_calling_agent(llm, tools=tools,prompt=controller_prompt)

In [276]:
response_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a response agent for a medical chatbot. Your job is to give an appopriate response to the user's medical query. If the user's prompt is not medical related that do not answer and just reply that you are a medical chatbot.
"""),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")  # This is the correct role for tool responses or internal reasoning
])

In [277]:
response_agent = create_tool_calling_agent(llm, tools=tools,prompt=response_prompt)

In [278]:
import uuid



def RAG_Agent(user_prompt):
  data={"id":user_info['user_id'],"text":user_prompt}
  x = pc.inference.embed(
    model="llama-text-embed-v2",
    inputs=[user_prompt],
    parameters={
        "input_type": "query"
    })

  vectors = []
  new_id = str(uuid.uuid4())

  vectors.append({
        "id": new_id,
        "values": e['values'],
        "metadata": {'user_id':user_info['user_id'],'text':user_prompt}
    })
  results = index.query(
    namespace="ns1",
    vector=x[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True,
    filter={"user_id":123} )
  if results["matches"]:
    return results
  else:
    return "None"

results=RAG_Agent(user_query)
if results!="None":
  context_texts = [match['metadata']['text'] for match in results['matches']]
  combined_context = "\n".join(context_texts)
  formatted_input = f"user: {user_query}\ncontext: {combined_context}"
else:
  formatted_input = f"user: {user_query}\ncontext:None"
print(formatted_input)



user: My nose is bleeding
context:None


In [279]:
from langchain.prompts import ChatPromptTemplate

context_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a checker agent for a medical chatbot.

A Retrieval-Augmented Generation (RAG) system has fetched the top context documents for a given user query. If no context was fetched than context will be False
Your task is to review the query and the retrieved context, then determine whether additional information is needed from the user.

If the current context is not sufficient to answer the user's question accurately, respond by listing the specific missing information you would ask the user for.

If the context is sufficient, respond with:
"Sufficient"

The input format is as follows:
user: <user_query>
context: <retrieved_context>or None if no context


"""),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])




In [280]:
context_agent = create_tool_calling_agent(llm, tools=tools,prompt=context_prompt)

In [281]:
from langchain.agents import AgentExecutor

control_agent_executor = AgentExecutor(agent=controller_agent, tools=tools, verbose=True)

response_agent_executor = AgentExecutor(agent=response_agent, tools=tools, verbose=True)

context_agent_executor = AgentExecutor(agent=context_agent, tools=tools, verbose=True)

In [282]:
user_query="My nose is bleeding"
selected_agent=agent_executor.invoke({"input":user_query})
selected_agent



> Entering new AgentExecutor chain...
retriever_agent

> Finished chain.


{'input': 'My nose is bleeding', 'output': 'retriever_agent'}

In [283]:
print(selected_agent)

{'input': 'My nose is bleeding', 'output': 'retriever_agent'}


In [284]:
if selected_agent['output'] == "Response_agent":
    result = response_agent_executor.invoke({"input": user_query})
else:
  results=RAG_Agent(user_query)
  formatted_query=''
  if results!="None":
    context_texts = [match['metadata']['text'] for match in results['matches']]
    combined_context = "\n".join(context_texts)
    formatted_query = f"user: {user_query}\ncontext: {combined_context}"
  else:
    formatted_query = f"user: {user_query}\ncontext:None"
  print(formatted_query)
  contexts=context_agent_executor.invoke({"input":formatted_query})
  context=''
  if contexts['output']!='Sufficient':
    context=input()
  final_query=user_query+formatted_query+context
  result = response_agent_executor.invoke({"input":final_query})
  result['output']






user: My nose is bleeding
context:None


> Entering new AgentExecutor chain...
I need more information to provide an accurate response. Please provide the following details:

1. How long has your nose been bleeding?
2. Is the bleeding heavy or light?
3. Have you experienced any trauma or injury to your nose recently?
4. Do you have any underlying medical conditions, such as high blood pressure or bleeding disorders?
5. Have you taken any medications recently that may be causing the bleeding?

Once I have this information, I can better assist you.

> Finished chain.
skss


> Entering new AgentExecutor chain...
A nosebleed can be caused by several factors, including dry air, blowing your nose too hard, or an injury to the nose. To stop the bleeding, try the following steps:

1. Sit up straight and lean forward to prevent blood from flowing down the back of your throat.
2. Pinch the soft part of your nose shut with your thumb and index finger for 5-10 minutes. This will help to constrict 